### **Config**

In [1]:
import os
import sys

sys.path.insert(0, '/home/marco/epfl/magma/')
import config

In [2]:
# General configurations

MODEL = 'pegasus'

RE_SPLITTER = '\.(?!\d)|\n'     # do we split sentences of paragraphs?
                                # use '\.|\n' or '\n', respectively

# Output path
OUTPUT_PATH = config.MAGMA_DIR+'datasets/merge_or_chunk/'+MODEL+'/'
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

### **Init**

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import re
import pandas as pd
import gensim

if 'pegasus' in MODEL:
    from transformers import PegasusTokenizer
    tokenizer =\
        PegasusTokenizer.from_pretrained('google/pegasus-large')
elif 'bart' in MODEL:
    from transformers import BartTokenizer
    tokenizer =\
        BartTokenizer.from_pretrained('facebook/bart-large-cnn')
elif 't5' in MODEL:
    from transformers import T5Tokenizer
    tokenizer=\
        T5Tokenizer.from_pretrained('t5-large')

### **Karger Books Base Dataset**

In [6]:
base_dataset = config.MAGMA_DIR+'datasets/karger_books_base/df.csv'
df = pd.read_csv(base_dataset)
df = df.set_index(['book', 'chapter', 'section', 'subsection'])
df.bullets = df.bullets.map(eval, na_action='ignore')

### **Preprocessing**

#### Preprocessing

* Split based on RE_SPLITTER
* Explode the dataset
* Remove unwanted chars at beginning or end of sentence
* Remove multiple spaces
* Remove long words (> config.TOKEN_MAX_LEN chars)
* Remove short sentences / paragraphs (< config.PARA_MIN_LEN tokens)

In [7]:
# Split in sentences / paragraphs based on RE_SPLITTER
df.text =\
    df.text.map(lambda x: [p.strip() for p in re.split(RE_SPLITTER, x) if p!=''],
                na_action='ignore')
    
# explode to get one row for each paragraph /sentence
df = df.explode('text')
df = df.rename(columns={'text': 'para'})
df = df.dropna()

# Remove unwanted chars at beginning or end of sentence
df.para = df.para.map(lambda p: p.lstrip('.,;:-)] \n'))
df.para = df.para.map(lambda p: p.rstrip('.,;:-([ \n'))

# Remove multiple spaces
df.para = df.para.map(lambda p:
    re.sub('\s+', ' ', p).strip())

# Remove long words (> config.TOKEN_MAX_LEN chars)
def para2words(para):
    return gensim.utils.simple_preprocess(
        para, deacc=True, max_len=config.TOKEN_MAX_LEN)
df['para_proc'] = df.para.map(para2words)

# Remove short sentences / paragraphs (< config.PARA_MIN_LEN tokens)
df.loc[df.para_proc.map(len) <\
    config.PARA_MIN_LEN, 'para_proc'] = np.nan

df = df.dropna()

### **Merge or Chunk Sections**

In [8]:
df = df.groupby(level=[0,1,2,3], sort=False).agg(
    {'para': lambda t: ''.join([p+'. ' for p in t]),
     'bullets': lambda b: list(b)[0]})
df = df.rename(columns={'para': 'text'})

In [9]:
df['text_enc'] = df.text.map(tokenizer.tokenize)
df['text_num_tokens'] = df.text_enc.map(len)

#### Chunk function

In [10]:
def chunk_text(text, num_tok):
    text_sent =\
        [sent.strip()+'.' for sent in re.split(RE_SPLITTER, text) if len(sent) > 1]

    # calculate number of tokens per sentence
    num_tok_sent = [len(tokenizer.tokenize(sent)) for sent in text_sent]
    
    # calculate chunk dimension to fit into model
    n = int(np.ceil(num_tok / config.MODEL_MAX_LEN))
    len_chunk = int(num_tok / n)
    # get a more uniform splitting to avoid splits
    # which are too short at the end
    if len_chunk+50 > config.MODEL_MAX_LEN:
        len_chunk = int(num_tok / (n+1))
    
    len_curr = 0
    text_curr = []
    text_chunk = []
    for te, len_sent in zip(text_sent, num_tok_sent):

        if len_curr + len_sent < len_chunk:
            text_curr.append(te)
            len_curr += len_sent

        elif len_curr + len_sent >= config.MODEL_MAX_LEN:
            text_chunk.append(text_curr)

            text_curr = [te]
            len_curr = len_sent

        else: # >= len_chunk && < MODEL_MAX_LEN
            text_curr.append(te)
            text_chunk.append(text_curr)
            
            text_curr = []
            len_curr = 0

    if len_curr > 0:
        text_chunk.append(text_curr)

    return text_chunk

#### Merge or Chunk

In [11]:
def chunk_or_merge(df_merge, num_row, num_tok, bid, cpt, i, index):
    print('num_tok MOC', num_tok)

    # if the text fits perfectly, we just add it to df_merge
    if num_tok >= CHUNK_MIN_LEN and num_tok < config.MODEL_MAX_LEN:
        id_merged = index[i-num_row:i]
        print('summ', id_merged)

        data_merged = []
        for c in df.columns:
            if c == 'bullets':
                data_merged.append(df.loc[id_merged, c].iloc[0])
            else:
                data_merged.append(df.loc[id_merged, c].sum())

        new_index = pd.MultiIndex.from_arrays(
            [[bid], [cpt], [str(id_merged)], [False]])
        df_merge_add = pd.DataFrame(
            data = [data_merged],
            index = new_index,
            columns = df.columns)
        df_merge = df_merge.append(df_merge_add)
        return False, df_merge

    # if the text is too long, we chunk it
    elif num_tok >= config.MODEL_MAX_LEN:
        id_merged = index[i-num_row:i]
        print('chunking', id_merged)

        new_index = pd.MultiIndex.from_arrays(
            [[bid], [cpt], [str(id_merged)], [True]])

        text_merged = df.loc[id_merged, 'text'].sum()
        text_chunked = chunk_text(text_merged, num_tok)
        for tc in text_chunked:
            t = ' '.join(tc)
            df_merge_add = pd.DataFrame(
                data = [[t,
                        df.iloc[i-1].bullets,
                        tokenizer.tokenize(t),
                        len(tokenizer.tokenize(t))]],
                index = new_index,
                columns = df.columns)
            df_merge = df_merge.append(df_merge_add)
        return False, df_merge

    # if the text is too short, we merge it
    elif num_tok < CHUNK_MIN_LEN:
        print('skipping', index[i-num_row:i])
        return True, df_merge

#### Create the new dataset

In [12]:
# define new dataset for the chapters
indices = pd.MultiIndex.from_arrays(
    [[], [], [], []],
    names = ['book', 'chapter', 'merge', 'chunk'])
df_moc = pd.DataFrame(index = indices, columns = df.columns.tolist())

# minimum length of a chunk
CHUNK_MIN_LEN = np.ceil(config.BULLETS_MED_LEN / config.MAX_RATIO)
print('Chunk minimum length:', CHUNK_MIN_LEN)
print()

index = df.index.tolist()

i_cpt = 0
for bid in sorted(list(set(df.index.get_level_values(0)))):
    cpts = config.get_cpts(df, bid)
    for cpt in cpts:
        print('book', bid, 'chapter', cpt)
        print()

        merge = False
        num_row, num_row_past = 0, 0
        num_tok, num_tok_past = 0, 0
        max_i = i_cpt + len(df.loc[bid, cpt])

        # for each section in chapter
        for i in range(i_cpt, max_i):
            # if previous section is too short, 
            # we merge it with the one below
            if merge: 
                num_row += 1
                print('merging', num_tok, '+', df.iloc[i].text_num_tokens)
                num_tok += df.iloc[i].text_num_tokens
            else:
                num_row_past = num_row
                num_row = 1
                num_tok_past = num_tok
                num_tok = df.iloc[i].text_num_tokens
            '''
                after merging or considering a new section, we call this function
                to study the length. Then we either add the row to the dataset, chunk it 
                or do nothing if it is still too short
            '''
            merge, df_moc =\
                chunk_or_merge(df_moc, num_row, num_tok, bid, cpt, i+1, index)

            print()
            
            '''
                if the last chunk of the chapter is too small, we merge it
                with the above and repeat one last time the process
            '''
            if merge and (i+1 == max_i):
                print('EXCEPTION, last chunk too small.')
                if num_row_past == 0:
                    _, df_moc =\
                        chunk_or_merge(df_moc, num_row, config.MODEL_MAX_LEN-1, bid, cpt, i+1, index)
                else:
                    num_row += num_row_past
                    num_tok += num_tok_past
                    # drop last row added and re-calculate for new, bigger chunk
                    df_moc = df_moc.drop(df_moc.index[-1])
                    _, df_moc =\
                        chunk_or_merge(df_moc, num_row, num_tok, bid, cpt, i+1, index)
                 
        i_cpt += len(df.loc[bid, cpt])
        print('\n############################')
        print()
        print()

for idx, r in df_moc.groupby(level=[0,1,2,3]).text.apply(list).iteritems():
    idx_merged = eval(idx[2])
    old_txt = ''.join(df.loc[idx_merged].text.tolist()).strip()
    new_txt = ' '.join(r).strip()
    assert old_txt == new_txt

Chunk minimum length: 720.0

book 9781905832729 chapter ch_2

num_tok MOC 11
skipping [(9781905832729, 'ch_2', 'ch_2', 'ch_2')]

merging 11 + 239
num_tok MOC 250
skipping [(9781905832729, 'ch_2', 'ch_2', 'ch_2'), (9781905832729, 'ch_2', '1.i', '1.i')]

merging 250 + 692
num_tok MOC 942
summ [(9781905832729, 'ch_2', 'ch_2', 'ch_2'), (9781905832729, 'ch_2', '1.i', '1.i'), (9781905832729, 'ch_2', '1.ii', '1.ii')]


############################


book 9781905832729 chapter ch_3

num_tok MOC 93
skipping [(9781905832729, 'ch_3', '2.i', '2.i')]

merging 93 + 268
num_tok MOC 361
skipping [(9781905832729, 'ch_3', '2.i', '2.i'), (9781905832729, 'ch_3', '2.ii', '2.ii')]

merging 361 + 488
num_tok MOC 849
summ [(9781905832729, 'ch_3', '2.i', '2.i'), (9781905832729, 'ch_3', '2.ii', '2.ii'), (9781905832729, 'ch_3', '2.iii', '2.iii')]

num_tok MOC 424
skipping [(9781905832729, 'ch_3', '2.iv', '2.iv')]

merging 424 + 867
num_tok MOC 1291
chunking [(9781905832729, 'ch_3', '2.iv', '2.iv'), (978190583272

/home/marco/miniconda3/envs/magma/lib/python3.6/site-packages/ipykernel_launcher.py:24: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/marco/miniconda3/envs/magma/lib/python3.6/site-packages/ipykernel_launcher.py:66: PerformanceWarning: indexing past lexsort depth may impact performance.




############################


book 9781905832729 chapter ch_4

num_tok MOC 8
skipping [(9781905832729, 'ch_4', 'ch_4', 'ch_4')]

merging 8 + 240
num_tok MOC 248
skipping [(9781905832729, 'ch_4', 'ch_4', 'ch_4'), (9781905832729, 'ch_4', '3.i', '3.i')]

merging 248 + 339
num_tok MOC 587
skipping [(9781905832729, 'ch_4', 'ch_4', 'ch_4'), (9781905832729, 'ch_4', '3.i', '3.i'), (9781905832729, 'ch_4', '3.ii', '3.ii')]

merging 587 + 287
num_tok MOC 874
summ [(9781905832729, 'ch_4', 'ch_4', 'ch_4'), (9781905832729, 'ch_4', '3.i', '3.i'), (9781905832729, 'ch_4', '3.ii', '3.ii'), (9781905832729, 'ch_4', '3.iii', '3.iii')]

num_tok MOC 62
skipping [(9781905832729, 'ch_4', '3.iv', '3.iv')]

merging 62 + 164
num_tok MOC 226
skipping [(9781905832729, 'ch_4', '3.iv', '3.iv'), (9781905832729, 'ch_4', '3.v', '3.v')]

merging 226 + 2193
num_tok MOC 2419
chunking [(9781905832729, 'ch_4', '3.iv', '3.iv'), (9781905832729, 'ch_4', '3.v', '3.v'), (9781905832729, 'ch_4', '3.vi', '3.vi')]

num_tok MOC 615


############################


book 9781908541024 chapter ch_10

num_tok MOC 194
skipping [(9781908541024, 'ch_10', 'ch_10', 'ch_10')]

merging 194 + 93
num_tok MOC 287
skipping [(9781908541024, 'ch_10', 'ch_10', 'ch_10'), (9781908541024, 'ch_10', '9.i', '9.i')]

merging 287 + 201
num_tok MOC 488
skipping [(9781908541024, 'ch_10', 'ch_10', 'ch_10'), (9781908541024, 'ch_10', '9.i', '9.i'), (9781908541024, 'ch_10', '9.ii', '9.ii')]

merging 488 + 493
num_tok MOC 981
summ [(9781908541024, 'ch_10', 'ch_10', 'ch_10'), (9781908541024, 'ch_10', '9.i', '9.i'), (9781908541024, 'ch_10', '9.ii', '9.ii'), (9781908541024, 'ch_10', '9.iii', '9.iii')]

num_tok MOC 3048
chunking [(9781908541024, 'ch_10', '9.iv', '9.iv')]

num_tok MOC 322
skipping [(9781908541024, 'ch_10', '9.v', '9.v')]

merging 322 + 190
num_tok MOC 512
skipping [(9781908541024, 'ch_10', '9.v', '9.v'), (9781908541024, 'ch_10', '9.vi', '9.vi')]

merging 512 + 178
num_tok MOC 690
skipping [(9781908541024, 'ch_10', '9.v', '9.v'), (9781



############################


book 9781908541062 chapter ch_7

num_tok MOC 51
skipping [(9781908541062, 'ch_7', 'ch_7', 'ch_7')]

merging 51 + 854
num_tok MOC 905
summ [(9781908541062, 'ch_7', 'ch_7', 'ch_7'), (9781908541062, 'ch_7', '6.i', '6.i')]

num_tok MOC 882
summ [(9781908541062, 'ch_7', '6.ii', '6.ii')]

num_tok MOC 226
skipping [(9781908541062, 'ch_7', '6.iii', '6.iii')]

merging 226 + 263
num_tok MOC 489
skipping [(9781908541062, 'ch_7', '6.iii', '6.iii'), (9781908541062, 'ch_7', '6.iv', '6.iv')]

EXCEPTION, last chunk too small.
num_tok MOC 1371
chunking [(9781908541062, 'ch_7', '6.ii', '6.ii'), (9781908541062, 'ch_7', '6.iii', '6.iii'), (9781908541062, 'ch_7', '6.iv', '6.iv')]

############################


book 9781908541062 chapter ch_8

num_tok MOC 157
skipping [(9781908541062, 'ch_8', 'ch_8', 'ch_8')]

merging 157 + 502
num_tok MOC 659
skipping [(9781908541062, 'ch_8', 'ch_8', 'ch_8'), (9781908541062, 'ch_8', '7.i', '7.i')]

merging 659 + 2082
num_tok MOC 2741
chunk

...


num_tok MOC 1261
chunking [(9781912776276, 'chp4', 'sec17', 'sec17')]

num_tok MOC 525
skipping [(9781912776276, 'chp4', 'sec18', 'sec18')]

EXCEPTION, last chunk too small.
num_tok MOC 1786
chunking [(9781912776276, 'chp4', 'sec17', 'sec17'), (9781912776276, 'chp4', 'sec18', 'sec18')]

############################


book 9781912776276 chapter chp5

num_tok MOC 439
skipping [(9781912776276, 'chp5', 'sec19', 'sec19')]

merging 439 + 436
num_tok MOC 875
summ [(9781912776276, 'chp5', 'sec19', 'sec19'), (9781912776276, 'chp5', 'sec20', 'sec20')]

num_tok MOC 3999
chunking [(9781912776276, 'chp5', 'sec21', 'sec21')]

num_tok MOC 231
skipping [(9781912776276, 'chp5', 'sec22', 'sec22')]

merging 231 + 277
num_tok MOC 508
skipping [(9781912776276, 'chp5', 'sec22', 'sec22'), (9781912776276, 'chp5', 'sec23', 'sec23')]

merging 508 + 133
num_tok MOC 641
skipping [(9781912776276, 'chp5', 'sec22', 'sec22'), (9781912776276, 'chp5', 'sec23', 'sec23'), (9781912776276, 'chp5', 'sec24', 'sec24')]

merg



############################


book 9781912776726 chapter ch2

num_tok MOC 154
skipping [(9781912776726, 'ch2', 'ch2', 'ch2')]

merging 154 + 421
num_tok MOC 575
skipping [(9781912776726, 'ch2', 'ch2', 'ch2'), (9781912776726, 'ch2', 'sec5', 'sec5')]

merging 575 + 572
num_tok MOC 1147
chunking [(9781912776726, 'ch2', 'ch2', 'ch2'), (9781912776726, 'ch2', 'sec5', 'sec5'), (9781912776726, 'ch2', 'sec6', 'sec6')]


############################


book 9781912776726 chapter ch3

num_tok MOC 463
skipping [(9781912776726, 'ch3', 'sec7', 'sec7')]

merging 463 + 533
num_tok MOC 996
summ [(9781912776726, 'ch3', 'sec7', 'sec7'), (9781912776726, 'ch3', 'sec8', 'sec8')]

num_tok MOC 3583
chunking [(9781912776726, 'ch3', 'sec9', 'sec9')]

num_tok MOC 252
skipping [(9781912776726, 'ch3', 'sec10', 'sec10')]

EXCEPTION, last chunk too small.
num_tok MOC 3835
chunking [(9781912776726, 'ch3', 'sec9', 'sec9'), (9781912776726, 'ch3', 'sec10', 'sec10')]

############################


book 9781912776726 c


############################


book 9783318066241 chapter ch4

num_tok MOC 190
skipping [(9783318066241, 'ch4', 'ch4', 'ch4')]

merging 190 + 81
num_tok MOC 271
skipping [(9783318066241, 'ch4', 'ch4', 'ch4'), (9783318066241, 'ch4', 'sec11', 'sec11')]

merging 271 + 854
num_tok MOC 1125
chunking [(9783318066241, 'ch4', 'ch4', 'ch4'), (9783318066241, 'ch4', 'sec11', 'sec11'), (9783318066241, 'ch4', 'sec12', 'sec12')]

num_tok MOC 439
skipping [(9783318066241, 'ch4', 'sec13', 'sec13')]

EXCEPTION, last chunk too small.
num_tok MOC 1564
chunking [(9783318066241, 'ch4', 'ch4', 'ch4'), (9783318066241, 'ch4', 'sec11', 'sec11'), (9783318066241, 'ch4', 'sec12', 'sec12'), (9783318066241, 'ch4', 'sec13', 'sec13')]

############################


book 9783318066241 chapter ch5

num_tok MOC 150
skipping [(9783318066241, 'ch5', 'ch5', 'ch5')]

merging 150 + 799
num_tok MOC 949
summ [(9783318066241, 'ch5', 'ch5', 'ch5'), (9783318066241, 'ch5', 'sec14', 'sec14')]

num_tok MOC 401
skipping [(978331806


num_tok MOC 229
skipping [(9783318066685, 'ch3', 'sec39', 'sec43_sub_sec39')]

merging 229 + 208
num_tok MOC 437
skipping [(9783318066685, 'ch3', 'sec39', 'sec43_sub_sec39'), (9783318066685, 'ch3', 'sec39', 'sec44_sub_sec39')]

merging 437 + 183
num_tok MOC 620
skipping [(9783318066685, 'ch3', 'sec39', 'sec43_sub_sec39'), (9783318066685, 'ch3', 'sec39', 'sec44_sub_sec39'), (9783318066685, 'ch3', 'sec39', 'sec45_sub_sec39')]

merging 620 + 234
num_tok MOC 854
summ [(9783318066685, 'ch3', 'sec39', 'sec43_sub_sec39'), (9783318066685, 'ch3', 'sec39', 'sec44_sub_sec39'), (9783318066685, 'ch3', 'sec39', 'sec45_sub_sec39'), (9783318066685, 'ch3', 'sec39', 'sec46_sub_sec39')]

num_tok MOC 328
skipping [(9783318066685, 'ch3', 'sec47', 'sec47')]

merging 328 + 270
num_tok MOC 598
skipping [(9783318066685, 'ch3', 'sec47', 'sec47'), (9783318066685, 'ch3', 'sec48', 'sec48')]

merging 598 + 284
num_tok MOC 882
summ [(9783318066685, 'ch3', 'sec47', 'sec47'), (9783318066685, 'ch3', 'sec48', 'sec48'),


############################


book 9783318067095 chapter ch8

num_tok MOC 485
skipping [(9783318067095, 'ch8', 'ch8', 'ch8')]

merging 485 + 745
num_tok MOC 1230
chunking [(9783318067095, 'ch8', 'ch8', 'ch8'), (9783318067095, 'ch8', 'sec25', 'sec25')]

num_tok MOC 281
skipping [(9783318067095, 'ch8', 'sec26', 'sec26')]

EXCEPTION, last chunk too small.
num_tok MOC 1511
chunking [(9783318067095, 'ch8', 'ch8', 'ch8'), (9783318067095, 'ch8', 'sec25', 'sec25'), (9783318067095, 'ch8', 'sec26', 'sec26')]

############################


book 9783318067095 chapter ch9

num_tok MOC 56
skipping [(9783318067095, 'ch9', 'ch9', 'ch9')]

merging 56 + 1468
num_tok MOC 1524
chunking [(9783318067095, 'ch9', 'ch9', 'ch9'), (9783318067095, 'ch9', 'sec27', 'sec27')]

num_tok MOC 152
skipping [(9783318067095, 'ch9', 'sec28', 'sec28')]

merging 152 + 291
num_tok MOC 443
skipping [(9783318067095, 'ch9', 'sec28', 'sec28'), (9783318067095, 'ch9', 'sec29', 'sec29')]

EXCEPTION, last chunk too small.
num_tok M

In [13]:
df_moc = df_moc.reset_index(level=[2, 3]).groupby(['book', 'chapter'], sort=False).agg({
    'merge': lambda m: list(m),
    'chunk': lambda c: list(c),
    'text': lambda t: list(t),
    'bullets': lambda b: list(b)[0]
})

#### Save new dataset

In [14]:
df_moc.to_csv(OUTPUT_PATH+'df.csv')

#### Create train, test, validation

Generate files:
* train.source
* train.target
* val.source
* val.target
* test.source
* test.target

Since we do not have a lot of samples, validation and test will be equal for us.

In [15]:
df_moc.bullets = df_moc.bullets.map(lambda b: ' '.join(b))

In [16]:
df_moc = df_moc.sample(frac=1, random_state=config.SEED)
df_moc['num_chunks'] = df_moc.text.map(len).cumsum()
tot_chunk = df_moc.num_chunks.iloc[-1]
split1 = np.where(df_moc.num_chunks > int(tot_chunk*0.8))[0][0]+1
split2 = np.where(df_moc.num_chunks > int(tot_chunk*0.9))[0][0]+1
print(split1, split2)

365 409


In [17]:
train, val, test =\
    df_moc.iloc[:split1].explode('text'),\
    df_moc.iloc[split1:split2].explode('text'),\
    df_moc.iloc[split2:].explode('text')

train.to_csv(OUTPUT_PATH+'train.csv')
val.to_csv(OUTPUT_PATH+'val.csv')
test.to_csv(OUTPUT_PATH+'test.csv')

In [18]:
with open(OUTPUT_PATH+'train.source', 'w') as tr_s,\
    open(OUTPUT_PATH+'train.target', 'w') as tr_t,\
    open(OUTPUT_PATH+'train.index', 'w') as tr_i:
    for idx, row in train[['text', 'bullets']].iterrows():
        tr_i.write(str(idx) + '\n')
        tr_s.write(row.text + '\n')
        tr_t.write(row.bullets + '\n')

In [19]:
with open(OUTPUT_PATH+'val.source', 'w') as va_s,\
    open(OUTPUT_PATH+'val.target', 'w') as va_t,\
    open(OUTPUT_PATH+'val.index', 'w') as va_i:
    for idx, row in val[['text', 'bullets']].iterrows():
        va_i.write(str(idx) + '\n')
        va_s.write(row.text + '\n')
        va_t.write(row.bullets + '\n')

In [20]:
with open(OUTPUT_PATH+'test.source', 'w') as te_s,\
    open(OUTPUT_PATH+'test.target', 'w') as te_t,\
    open(OUTPUT_PATH+'test.index', 'w') as te_i:
    for idx, row in test[['text', 'bullets']].iterrows():
        te_i.write(str(idx) + '\n')
        te_s.write(row.text + '\n')
        te_t.write(row.bullets + '\n')